In [110]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='ignore') 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
bus = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/천안시 데이터 시각화 프로젝트/around_bus_info_2km_08152321.pickle')
bus.head(3)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9,주변시설의총합
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,...,"{'documents': [], 'meta': {'is_end': True, 'pa...",{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 석곡...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,654
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,...,"{'documents': [], 'meta': {'is_end': True, 'pa...",{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 석곡...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,{'documents': [{'address_name': '충남 아산시 둔포면 둔포...,673
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 송화...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 남산...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 안정...,{'documents': [{'address_name': '경기 평택시 팽성읍 송화...,691


In [12]:
addressCopy = bus.copy()

headers = {'Authorization': 'KakaoAK {}'.format('masking') }
x = addressCopy.at[0,'경도']
y =  addressCopy.at[0,'위도']

searching ='x={x}&y={y}'.format(x=str(x),y=str(y))


url='https://dapi.kakao.com/v2/local/geo/coord2address.json?{}'.format(searching)

data = requests.post(url,headers=headers)
result = json.loads(data.text)
documents = result['documents']
# documents
result


{'meta': {'total_count': 1},
 'documents': [{'road_address': None,
   'address': {'address_name': '경기 평택시 팽성읍 석근리 191-1',
    'region_1depth_name': '경기',
    'region_2depth_name': '평택시',
    'region_3depth_name': '팽성읍 석근리',
    'mountain_yn': 'N',
    'main_address_no': '191',
    'sub_address_no': '1',
    'zip_code': ''}}]}

In [17]:
documents[0]

{'road_address': None,
 'address': {'address_name': '경기 평택시 팽성읍 석근리 191-1',
  'region_1depth_name': '경기',
  'region_2depth_name': '평택시',
  'region_3depth_name': '팽성읍 석근리',
  'mountain_yn': 'N',
  'main_address_no': '191',
  'sub_address_no': '1',
  'zip_code': ''}}

In [23]:
addressCopy = bus.copy()

headers = {'Authorization': 'KakaoAK {}'.format('masking') }

addressCopy['coord2address'] = None

cnt=0 
for idx in tqdm(range(len(addressCopy))):
  try:

    x = addressCopy.at[idx,'경도']
    y =  addressCopy.at[idx,'위도']
    searching ='x={x}&y={y}'.format(x=str(x),y=str(y))
    url='https://dapi.kakao.com/v2/local/geo/coord2address.json?{}'.format(searching)
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    documents = result['documents']
    
    addressCopy.at[idx,'coord2address'] = documents[0]
  except:
    cnt+=1
    if cnt%50==0:
      print('쿼리값을 못찾는중:',cnt) 

    pass


100%|██████████| 4632/4632 [08:29<00:00,  9.10it/s]


In [35]:
# addressCopy.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/천안시 데이터 시각화 프로젝트/bus_coord2address.pickle')

36.993883	127.089649	

정류장 : 신평11통(유천3리)
구글좌표 : 36.9731 127.1067
원본좌효 : 36.973053 127.106688
원본 분류 : 천안시
카카오 : 경기도 평택시 유천동

정류장명 : 가산1리
구글좌표 : 36.936979 127.194877
원본좌표 : 36.936943 127.194741

In [ ]:
# 신평11통(유천3리)
addressCopy[addressCopy['도시명']=='천안시'].sample(10)

In [97]:
addressCopy[addressCopy['정류장명']=='신평11통(유천3리)']
# addressCopy.loc[addressCopy[addressCopy['정류장명'] == '신평11통(유천3리)'].index[0], 'coord2address']

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9,주변시설의총합,coord2address
2140,CAB214000033,신평11통(유천3리),36.973053,127.106688,20221012050102,33.0,34010,천안시,천안,{'documents': [{'address_name': '경기 평택시 합정동 96...,...,{'documents': [{'address_name': '경기 평택시 합정동 91...,{'documents': [{'address_name': '경기 평택시 비전동 84...,{'documents': [{'address_name': '경기 평택시 합정동 95...,"{'documents': [], 'meta': {'is_end': True, 'pa...",{'documents': [{'address_name': '경기 평택시 유천동 48...,{'documents': [{'address_name': '경기 평택시 유천동 45...,{'documents': [{'address_name': '경기 평택시 합정동 96...,{'documents': [{'address_name': '경기 평택시 합정동 96...,419,"{'road_address': None, 'address': {'address_na..."


{'road_address': None, 'address': {'address_name': '충남 천안시 서북구 입장면 가산리 320-7', 'region_1depth_name': '충남', 'region_2depth_name': '천안시 서북구', 'region_3depth_name': '입장면 가산리', 'mountain_yn': 'N', 'main_address_no': '320', 'sub_address_no': '7', 'zip_code': ''}}

In [111]:
addressCopy.at[0,'coord2address']
addressCopy.at[0,'coord2address']['address']['address_name']


{'road_address': None,
 'address': {'address_name': '경기 평택시 팽성읍 석근리 191-1',
  'region_1depth_name': '경기',
  'region_2depth_name': '평택시',
  'region_3depth_name': '팽성읍 석근리',
  'mountain_yn': 'N',
  'main_address_no': '191',
  'sub_address_no': '1',
  'zip_code': ''}}

'경기 평택시 팽성읍 석근리 191-1'

In [104]:
addressCopy2 = addressCopy.copy()

# def extract_region_2depth_name(row):
#     return row['address']['region_2depth_name']

def extract_region_name(row):
    if row and 'address' in row and 'region_2depth_name' in row['address']:
        return row['address']['region_2depth_name']
    else:
        return None  # Return a default value if keys are missing

addressCopy2['region_2depth_name'] = addressCopy2['coord2address'].apply(extract_region_name)
addressCopy2['region_2depth_name']

0       평택시
1       평택시
2       평택시
3       평택시
4       평택시
       ... 
4627    아산시
4628    아산시
4629    아산시
4630    아산시
4631    아산시
Name: region_2depth_name, Length: 4632, dtype: object

In [105]:
addressCopy2['region_2depth_name'].value_counts()

아산시        2010
천안시 동남구    1237
천안시 서북구    1077
평택시          79
세종시          59
공주시          49
안성시          46
진천군          31
청주시 청원구       8
당진시           2
예산군           2
Name: region_2depth_name, dtype: int64

In [ ]:
target_list = ['아산시','천안시 동남구','천안시 서북구']
addressCopy3 = addressCopy2[addressCopy2['region_2depth_name'].isin(target_list)]
addressCopy3

In [112]:
def extract_region_name(row):
    if row and 'address' in row and 'address_name' in row['address']:
        return row['address']['address_name']
    else:
        return None  # Return a default value if keys are missing

addressCopy3['도로명주소'] = addressCopy3['coord2address'].apply(extract_region_name)
addressCopy3['도로명주소']

39      충남 천안시 서북구 성환읍 성환리 675-1
40       충남 천안시 서북구 성환읍 우신리 24-5
41       충남 천안시 서북구 성환읍 신방리 1-10
42      충남 천안시 서북구 성환읍 성환리 482-1
43      충남 천안시 서북구 성환읍 성환리 372-4
                  ...           
4627        충남 아산시 둔포면 시포리 183-1
4628        충남 아산시 둔포면 둔포리 417-2
4629        충남 아산시 둔포면 둔포리 417-2
4630        충남 아산시 둔포면 둔포리 462-3
4631        충남 아산시 둔포면 둔포리 477-8
Name: 도로명주소, Length: 4324, dtype: object

In [118]:
addressCopy3.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/around_bus_info_2km_08161152.pickle')

In [119]:
addressCopy3.to_csv('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/around_bus_info_2km_08161152.csv',encoding='cp949',index=False)


In [121]:
road_csv = pd.read_csv('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/around_bus_info_2km_08161152.csv',encoding='cp949')
road_csv.head()

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,MT1,...,AT4,AD5,FD6,CE7,HP8,PM9,주변시설의총합,coord2address,region_2depth_name,도로명주소
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,550,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 성환리 675-1
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,515,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 우신리 24-5
2,ASB285000079,국립축산과학원,36.925931,127.111136,20221012050123,79.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,245,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 신방리 1-10
3,ASB285000094,성환8리,36.921479,127.122505,20221012050123,94.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,539,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 성환리 482-1
4,ASB285000095,성환터미널,36.917221,127.132732,20221012050123,95.0,34040,아산시,아산,{'documents': [{'address_name': '충남 천안시 서북구 성환...,...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,{'documents': [{'address_name': '충남 천안시 서북구 성환...,620,"{'road_address': None, 'address': {'address_na...",천안시 서북구,충남 천안시 서북구 성환읍 성환리 372-4


In [123]:
road_csv['region_2depth_name'].value_counts()

아산시        2010
천안시 동남구    1237
천안시 서북구    1077
Name: region_2depth_name, dtype: int64

In [91]:
# addressCopy.to_csv('/Users/iseunghyeon/Desktop/프로젝트/천안시 데이터 시각화 프로젝트/bus_coord2addressView.csv',index=False,encoding='cp949')